# Does model calibration differ with nodule/lung cancer subtype?

In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None
import os
import glob
from os import path
import numpy as np

In [3]:
root_dir = "/nfs/masi/NLST/package-nlst-7-2018.09.24"
raw_cli = path.join(root_dir, "participant.data.d100517.csv")
raw_df = pd.read_csv(raw_cli, dtype={"pid": str})

type_path = os.path.join(root_dir, "Lung Cancer", "lung_cancer.data.d100517.csv")
type_df = pd.read_csv(type_path, dtype={'pid': str})
type_df

/tmp/ipykernel_738457/1396119642.py:3: DtypeWarning: Columns (99,248,249) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_df = pd.read_csv(raw_cli, dtype={"pid": str})


,dataset_version,pid,candx_days,clinical_m,clinical_m_7thed,clinical_n,clinical_n_7thed,clinical_stag,clinical_t,clinical_t_7thed,...,path_n_7thed,path_stag,path_t,path_t_7thed,source_best_stage,stage_only,stage_sum,study_yr,topog_source,valcsg
0,2011.02.03/10.05.17,100012,454,2.0,2.0,2.0,2.0,3,4.0,2.0,...,2.0,3,4.0,2.0,1,99,NaN,1,2.0,NaN
1,2011.02.03/10.05.17,100049,2428,2.0,2.0,2.0,2.0,7,6.0,7.0,...,NaN,99,NaN,NaN,2,7,1.0,6,1.0,NaN
2,2011.02.03/10.05.17,100055,148,3.0,3.0,4.0,4.0,10,5.0,5.0,...,NaN,99,NaN,NaN,2,10,3.0,0,NaN,NaN
3,2011.02.03/10.05.17,100147,491,2.0,2.0,2.0,2.0,3,4.0,3.0,...,2.0,3,4.0,3.0,1,3,1.0,1,2.0,NaN
4,2011.02.03/10.05.17,100158,795,2.0,2.0,2.0,2.0,3,4.0,2.0,...,2.0,3,4.0,2.0,1,99,NaN,2,2.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2145,2011.02.03/10.05.17,218824,27,2.0,2.0,3.0,3.0,7,5.0,5.0,...,2.0,4,5.0,5.0,1,4,1.0,0,2.0,3.0
2146,2011.02.03/10.05.17,218834,71,2.0,2.0,2.0,2.0,3,4.0,2.0,...,2.0,3,4.0,2.0,1,3,1.0,0,2.0,3.0
2147,2011.02.03/10.05.17,218838,1077,2.0,2.0,4.0,4.0,8,4.0,2.0,...,4.0,10,7.0,7.0,1,10,3.0,2,2.0,3.0
2148,2011.02.03/10.05.17,218873,1632,3.0,4.0,4.0,4.0,10,1.0,1.0,...,1.0,96,1.0,1.0,3,10,3.0,4,NaN,3.0


In [6]:
type_df['lc_morph'].value_counts(dropna=False)

8140    692
8070    438
8041    272
8046    250
8250    119
8012     50
8010     46
8000     40
8013     31
8071     27
8560     27
8252     21
8246     16
8550     16
8042     11
8260     10
8253     10
8240      9
8255      8
8045      8
8490      7
8481      7
8480      6
8072      5
8254      4
8033      3
8249      2
8980      2
8032      2
8323      2
8001      1
8570      1
8050      1
8021      1
8052      1
8022      1
8310      1
8084      1
8044      1
Name: lc_morph, dtype: int64

In [7]:
cohort_path = "/home/local/VANDERBILT/litz/github/MASILab/DeepLungScreening/cohorts/nlst_cohort_prep_v2.csv"
cohort_df = pd.read_csv(cohort_path, dtype={'pid':str})
merged = cohort_df.merge(type_df, on='pid')
merged

,pid,id,session,with_image,with_marker,age,gender,race,education,bmi,...,path_n_7thed,path_stag,path_t,path_t_7thed,source_best_stage,stage_only,stage_sum,study_yr,topog_source,valcsg
0,100012,100012time1999,0,True,True,61,2,1,6,22.240116,...,2.0,3,4.0,2.0,1,99,NaN,1,2.0,NaN
1,100012,100012time2000,1,True,True,61,2,1,6,22.240116,...,2.0,3,4.0,2.0,1,99,NaN,1,2.0,NaN
2,100147,100147time1999,0,True,True,68,1,1,1,29.953440,...,2.0,3,4.0,3.0,1,3,1.0,1,2.0,NaN
3,100147,100147time2000,1,True,True,68,1,1,1,29.953440,...,2.0,3,4.0,3.0,1,3,1.0,1,2.0,NaN
4,100158,100158time1999,0,True,True,65,1,1,4,28.737057,...,2.0,3,4.0,2.0,1,99,NaN,2,2.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2293,218750,218750time2000,1,True,True,55,2,1,5,19.485425,...,1.0,96,1.0,1.0,3,10,3.0,5,2.0,3.0
2294,218750,218750time2001,2,True,True,55,2,1,5,19.485425,...,1.0,96,1.0,1.0,3,10,3.0,5,2.0,3.0
2295,218762,218762time1999,0,True,True,55,2,1,5,20.967282,...,1.0,96,1.0,1.0,3,9,2.0,4,NaN,3.0
2296,218762,218762time2000,1,True,True,55,2,1,5,20.967282,...,1.0,96,1.0,1.0,3,9,2.0,4,NaN,3.0


In [11]:
merged.groupby('pid').max()

,id,session,with_image,with_marker,age,gender,race,education,bmi,copd,...,path_n_7thed,path_stag,path_t,path_t_7thed,source_best_stage,stage_only,stage_sum,study_yr,topog_source,valcsg
pid,,,,,,,,,,,,,,,,,,,,,
100012,100012time2000,1,True,True,61,2,1,6,22.240116,0.0,...,2.0,3,4.0,2.0,1,99,NaN,1,2.0,NaN
100147,100147time2000,1,True,True,68,1,1,1,29.953440,0.0,...,2.0,3,4.0,3.0,1,3,1.0,1,2.0,NaN
100158,100158time2001,2,True,True,65,1,1,4,28.737057,0.0,...,2.0,3,4.0,2.0,1,99,NaN,2,2.0,NaN
100242,100242time1999,0,True,True,60,2,1,4,30.896612,0.0,...,1.0,96,4.0,2.0,3,3,1.0,0,2.0,3.0
100280,100280time2000,1,True,True,60,2,1,5,23.913931,0.0,...,2.0,3,4.0,2.0,1,99,NaN,1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218499,218499time2001,2,True,True,63,1,1,2,23.110395,0.0,...,1.0,96,1.0,1.0,2,10,3.0,3,NaN,3.0
218510,218510time2001,2,True,True,64,1,1,6,27.802713,1.0,...,2.0,3,4.0,2.0,1,3,1.0,2,2.0,3.0
218705,218705time2001,2,True,True,68,1,1,4,37.305733,0.0,...,2.0,10,5.0,5.0,1,10,3.0,5,2.0,3.0


In [8]:
merged['lc_morph'].value_counts()

8140    745
8070    488
8041    279
8046    223
8250    176
8000     48
8010     43
8012     38
8560     34
8013     28
8252     26
8071     24
8042     16
8550     14
8246     12
8255     11
8260     10
8240      9
8045      9
8490      9
8480      9
8072      9
8254      7
8253      7
8481      6
8323      6
8032      3
8249      3
8084      3
8980      2
8570      1
Name: lc_morph, dtype: int64